In [1]:
import time
import math
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn import svm
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import confusion_matrix
from scipy.stats import friedmanchisquare

In [6]:
with open ('spambase_data.csv', 'w+') as file:
    for i in range(1, 58):
        file.write(str(str(i)+',').rstrip('\n'))
    file.write(str(str(58)).rstrip('\n'))
    file.write('\n')
    with open('spambase.data', 'r') as datafile:
        for line in datafile:
            file.write(line)

In [7]:
data = pd.read_csv('spambase_data.csv', delimiter = ',')
data = shuffle(data)
data_list = data.values.tolist()

In [8]:
data.head()

,1,2,3,4,5,6,7,8,9,10,...,49,50,51,52,53,54,55,56,57,58
3454,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.030,0.060,0.000,0.000,0.000,0.00,2.444,76,198,0
216,0.07,0.02,0.15,0.0,0.25,0.20,0.00,0.02,0.15,0.00,...,0.011,0.022,0.000,0.000,0.022,0.00,1.423,20,965,1
1684,0.00,0.26,0.78,0.0,0.26,0.26,0.08,1.04,0.52,1.56,...,0.096,0.234,0.000,0.358,0.261,0.11,3.554,54,981,1
3725,0.00,0.00,0.52,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.075,0.151,0.000,0.000,0.00,2.416,18,116,0
4114,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.00,0.00,...,0.000,0.188,0.000,0.564,0.000,0.00,1.000,1,14,0


In [9]:
data.shape

(4601, 58)

In [10]:
data.isnull().sum()

1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    0
20    0
21    0
22    0
23    0
24    0
25    0
26    0
27    0
28    0
29    0
30    0
31    0
32    0
33    0
34    0
35    0
36    0
37    0
38    0
39    0
40    0
41    0
42    0
43    0
44    0
45    0
46    0
47    0
48    0
49    0
50    0
51    0
52    0
53    0
54    0
55    0
56    0
57    0
58    0
dtype: int64

In [11]:
data_list = data.values.tolist()
data_input_list = data.iloc[:,:-1].values.tolist()
data_output_list = data.iloc[:,-1].values.tolist()

In [12]:
skf = StratifiedKFold(n_splits=10)
skf.get_n_splits(data_input_list,data_output_list)
StratifiedKFold(n_splits=10, random_state=None, shuffle=False)

StratifiedKFold(n_splits=10, random_state=None, shuffle=False)

In [13]:
Logistic_Regression = []
Support_Vector_Machine_classifier = []
Decision_Tree = []

Logistic_Regression_time = []
Support_Vector_Machine_classifier_time = []
Decision_Tree_time = []

Logistic_Regression_recall = []
Support_Vector_Machine_classifier_recall = []
Decision_Tree_recall = []

Logistic_Regression_precision = []
Support_Vector_Machine_classifier_precision = []
Decision_Tree_precision = []


Logistic_Regression_Fmeasure = []
Support_Vector_Machine_classifier_Fmeasure = []
Decision_Tree_Fmeasure = []

Logistic_Regression_ranks = []
Support_Vector_Machine_classifier_ranks = []
Decision_Tree_ranks = []


In [14]:
for train_index, test_index in skf.split(data_input_list,data_output_list):
    
    training_fold_list = []
    testing_fold_list = []
    training_fold_input_list = []
    training_fold_output_list = []
    testing_fold_input_list = []
    testing_fold_output_list = []
    
    for i in train_index:
        training_fold_list.append(data_list[i])
        training_fold_input_list.append(data_list[i][:-1])
        training_fold_output_list.append(data_list[i][-1])
        
    for j in test_index:
        testing_fold_list.append(data_list[j])
        testing_fold_input_list.append(data_list[j][:-1])
        testing_fold_output_list.append(data_list[j][-1])
        
    #training with logistic regression
    LG_Regression = LogisticRegression()
    time_start = time.clock() 
    LG_Regression.fit(training_fold_input_list,training_fold_output_list)
    time_elapsed = time.clock() - time_start
    LG_Regression_result = LG_Regression.predict(testing_fold_input_list)   
    Logistic_Regression_time.append(time_elapsed) 
    
    #training with SVM algorithm
    clf = svm.SVC(kernel='linear')
    time_start = time.clock()  
    clf.fit(training_fold_input_list,training_fold_output_list)
    time_elapsed = time.clock() - time_start
    svmresult_list = clf.predict(testing_fold_input_list)
    Support_Vector_Machine_classifier_time.append(time_elapsed)
    
    
    #training with Decision Tree
    D_clf = DecisionTreeClassifier(random_state=0)
    time_start = time.clock() 
    D_clf.fit(training_fold_input_list,training_fold_output_list)
    time_elapsed = time.clock() - time_start
    DS_result_list = D_clf.predict(testing_fold_input_list)
    Decision_Tree_time.append(time_elapsed)
    
    #calculating recall, precision and fmeasure for Logistic_Regression
    tn_lg, fp_lg, fn_lg, tp_lg = confusion_matrix(testing_fold_output_list,LG_Regression_result).ravel()
    Logistic_Regression.append((tp_lg+tn_lg)/(tn_lg + tp_lg + fn_lg + fp_lg))
    Logistic_Regression_recall.append(tp_lg/(tp_lg + fn_lg))
    Logistic_Regression_precision.append(tp_lg/(tp_lg + fp_lg))
    recall_1 = tp_lg/(tp_lg + fn_lg)
    precision_1 = tp_lg/(tp_lg + fp_lg)
    Logistic_Regression_Fmeasure.append((2*recall_1*precision_1)/(recall_1 + precision_1))
    
    #calculating recall, precision and fmeasure for SVM 
    tn_svm, fp_svm, fn_svm, tp_svm = confusion_matrix(testing_fold_output_list,svmresult_list).ravel()
    Support_Vector_Machine_classifier.append((tp_svm+tn_svm)/(tn_svm + tp_svm + fn_svm + fp_svm))
    Support_Vector_Machine_classifier_recall.append(tp_svm/(tp_svm + fn_svm))
    Support_Vector_Machine_classifier_precision.append(tp_svm/(tp_svm + fp_svm))
    recall_2 = tp_svm/(tp_svm + fn_svm)
    precision_2 = tp_svm/(tp_svm + fp_svm)
    Support_Vector_Machine_classifier_Fmeasure.append((2*recall_2*precision_2)/(recall_2 + precision_2))
    
    #calculating recall, precision and fmeasure for Decision_Tree
    tn_ds, fp_ds, fn_ds, tp_ds = confusion_matrix(testing_fold_output_list,DS_result_list).ravel()
    Decision_Tree.append((tp_ds+tn_ds)/(tn_ds + tp_ds + fn_ds + fp_ds))
    Decision_Tree_recall.append(tp_ds/(tp_ds + fn_ds))
    Decision_Tree_precision.append(tp_ds/(tp_ds + fp_ds))
    recall_3 = tp_ds/(tp_ds + fn_ds)
    precision_3 = tp_ds/(tp_ds + fp_ds)
    Decision_Tree_Fmeasure.append((2*recall_3*precision_3)/(recall_3 + precision_3))

In [30]:
def displayResults(x_1,x_2,x_3):
    print("Fold  ","  logistic regression   ","   SVM     ","  Decision Tree  ")
    for i in range(10):
       if i == 9:
           print (' {:1d}        {:0.4f}           {:0.4f}                {:0.4f}'.format(i+1, x_1[i],x_2[i],x_3[i]))
           continue
       print (' {:1d}         {:0.4f}           {:0.4f}                {:0.4f}'.format(i+1, x_1[i],x_2[i],x_3[i]))
    print()
    print (' avg       {:0.4f}           {:0.4f}                {:0.4f}'.format((np.array(x_1)).mean(),(np.array(x_2)).mean(),(np.array(x_3)).mean() ))
    print (' sd       {:0.4f}           {:0.4f}                {:0.4f}'.format((np.array(x_1)).std(), (np.array(x_2)).std(), (np.array(x_3)).std() ))
    print()
    print()
    print("Friedman test and results")
    print("Fold  ","  logistic regression   ","   SVM     ","  Decision Tree  ")
    for i in range(10):
        if i == 9:
            print (' {:1d}        {:0.4f}({:1d})           {:0.4f}({:1d})              {:0.4f}({:1d})'.format(i+1, x_1[i], Logistic_Regression_ranks[i],x_2[i],Support_Vector_Machine_classifier_ranks[i], x_3[i],Decision_Tree_ranks[i]))
            continue
        print (' {:1d}         {:0.4f}({:1d})           {:0.4f}({:1d})              {:0.4f}({:1d})'.format(i+1, x_1[i], Logistic_Regression_ranks[i],x_2[i],Support_Vector_Machine_classifier_ranks[i],x_3[i],Decision_Tree_ranks[i]))
    print()
    print (' avg rank    {:0.1f}                 {:0.1f}                    {:0.1f}'.format(sum(Logistic_Regression_ranks)/10, sum(Support_Vector_Machine_classifier_ranks)/10, sum(Decision_Tree_ranks)/10 ))
    print()
    z , y = friedmanchisquare(x_1,x_2,x_3)
    print(" Friedman statistic is {:0.4f}".format(z))
    if(z  > 7.8): #7.8 
        print(" Null Hypothesis is rejected: All the three algorithms don't perform equally")
        k = 3
        CD = 2.343 * math.sqrt((k * ( k + 1 ))/(6 * 10)) #k = 3 and n = 10
        print(' Critical Difference ',CD)
        print()
        if abs(sum(Logistic_Regression_ranks)/10 - sum(Support_Vector_Machine_classifier_ranks)/10) > CD:
            print(' difference between logistic regression and Support_Vector_Machine_classifier exceeds critical difference')
        if abs(sum(Support_Vector_Machine_classifier_ranks)/10 - sum(Decision_Tree_ranks)/10) > CD:
            print(' difference between Support_Vector_Machine_classifier and Decision_Tree exceeds critical difference')
        if abs(sum(Logistic_Regression_ranks)/10 - sum(Decision_Tree_ranks)/10) > CD:
            print(' difference between Logistic_Regression and Decision_Tree exceeds critical difference')
        print()
    else:
        print(" Failed to reject null Hypothesis : All the algorithms performs equally")

In [31]:
#Assigning ranks for measures
def rank(x_1,x_2,x_3,measure):
    if measure == 'Time':
        for i in range(10):
            if x_1[i] <= x_2[i]:
                if x_1[i] <= x_3[i]:
                    Logistic_Regression_ranks.append(1)
                    if x_2[i] <= x_3[i]:
                        Support_Vector_Machine_classifier_ranks.append(2)
                        Decision_Tree_ranks.append(3)
                    else:
                        Support_Vector_Machine_classifier_ranks.append(2)
                        Decision_Tree_ranks.append(3)
            if x_2[i] <= x_3[i]:
                if x_2[i] <= x_1[i]:
                    Support_Vector_Machine_classifier_ranks.append(1)
                    if x_1[i] <= x_3[i]:
                        Logistic_Regression_ranks.append(2)
                        Decision_Tree_ranks.append(3)
                    else:
                        Decision_Tree_ranks.append(2)
                        Logistic_Regression_ranks.append(3)
            if x_3[i] <= x_1[i]:
                if x_3[i] <= x_2[i]:
                    Decision_Tree_ranks.append(1)
                    if x_1[i] <= x_2[i]:
                        Logistic_Regression_ranks.append(2)
                        Support_Vector_Machine_classifier_ranks.append(3)
                    else:
                        Support_Vector_Machine_classifier_ranks.append(2)
                        Logistic_Regression_ranks.append(3)
    else:
        for i in range(10):
            if x_1[i] >= x_2[i]:
                if x_1[i] >= x_3[i]:
                    Logistic_Regression_ranks.append(1)
                    if x_2[i] >= x_3[i]:
                        Support_Vector_Machine_classifier_ranks.append(2)
                        Decision_Tree_ranks.append(3)
                    else:
                        Support_Vector_Machine_classifier_ranks.append(2)
                        Decision_Tree_ranks.append(3)
            if x_2[i] >= x_3[i]:
                if x_2[i] >= x_1[i]:
                    Support_Vector_Machine_classifier_ranks.append(1)
                    if x_1[i] >= x_3[i]:
                        Logistic_Regression_ranks.append(2)
                        Decision_Tree_ranks.append(3)
                    else:
                        Decision_Tree_ranks.append(2)
                        Logistic_Regression_ranks.append(3)
            if x_3[i] >= x_1[i]:
                if x_3[i] >= x_2[i]:
                    Decision_Tree_ranks.append(1)
                    if x_1[i] >= x_2[i]:
                        Logistic_Regression_ranks.append(2)
                        Support_Vector_Machine_classifier_ranks.append(3)
                    else:
                        Support_Vector_Machine_classifier_ranks.append(2)
                        Logistic_Regression_ranks.append(3)
    displayResults(x_1,x_2,x_3)

In [32]:
rank(Logistic_Regression, Support_Vector_Machine_classifier, Decision_Tree, 'Accuracy')
Logistic_Regression_ranks = []
Support_Vector_Machine_classifier_ranks = []
Decision_Tree_ranks = []

Fold     logistic regression       SVM        Decision Tree  
 1         0.9371           0.9328                0.9241
 2         0.9046           0.9132                0.8937
 3         0.9111           0.9154                0.9176
 4         0.9109           0.9130                0.8935
 5         0.9522           0.9587                0.9174
 6         0.9174           0.9217                0.9261
 7         0.9435           0.9543                0.9261
 8         0.9326           0.9239                0.9000
 9         0.9412           0.9390                0.9150
 10        0.9303           0.9325                0.9194

 avg       0.9281           0.9305                0.9133
 sd       0.0153           0.0155                0.0121


Friedman test and results
Fold     logistic regression       SVM        Decision Tree  
 1         0.9371(1)           0.9328(2)              0.9241(3)
 2         0.9046(2)           0.9132(1)              0.8937(3)
 3         0.9111(3)           0.915

In [33]:
rank(Logistic_Regression_Fmeasure, Support_Vector_Machine_classifier_Fmeasure, Decision_Tree_Fmeasure, 'F-measure')
Logistic_Regression_ranks = []
Support_Vector_Machine_classifier_ranks = []
Decision_Tree_ranks = []

Fold     logistic regression       SVM        Decision Tree  
 1         0.9197           0.9141                0.9072
 2         0.8778           0.8889                0.8693
 3         0.8852           0.8920                0.8944
 4         0.8832           0.8870                0.8658
 5         0.9382           0.9465                0.8950
 6         0.8902           0.8966                0.9050
 7         0.9282           0.9421                0.9056
 8         0.9132           0.9025                0.8708
 9         0.9244           0.9218                0.8926
 10        0.9111           0.9146                0.8992

 avg       0.9071           0.9106                0.8905
 sd       0.0203           0.0202                0.0151


Friedman test and results
Fold     logistic regression       SVM        Decision Tree  
 1         0.9197(1)           0.9141(2)              0.9072(3)
 2         0.8778(2)           0.8889(1)              0.8693(3)
 3         0.8852(3)           0.892

In [34]:
rank(Logistic_Regression_time, Support_Vector_Machine_classifier_time, Decision_Tree_time, 'Time')
Logistic_Regression_ranks = []
Support_Vector_Machine_classifier_ranks = []
Decision_Tree_ranks = []

Fold     logistic regression       SVM        Decision Tree  
 1         0.1317           151.3594                0.0533
 2         0.0491           294.3539                0.0528
 3         0.0870           175.3428                0.0477
 4         0.0474           303.0067                0.0552
 5         0.0573           213.2388                0.0556
 6         0.0730           144.3701                0.0524
 7         0.0722           604.8594                0.0517
 8         0.0781           153.9641                0.0548
 9         0.1022           162.3246                0.0505
 10        0.0360           221.0076                0.0459

 avg       0.0734           242.3827                0.0520
 sd       0.0271           132.5463                0.0030


Friedman test and results
Fold     logistic regression       SVM        Decision Tree  
 1         0.1317(2)           151.3594(3)              0.0533(1)
 2         0.0491(1)           294.3539(2)              0.0528(3)
 3      